In [25]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [27]:
features_df = pd.read_csv('train_x.csv')
labels_df = pd.read_csv('train_y.csv')
features_df.drop(columns=['Unnamed: 0'], inplace=True)
labels_df.drop(columns=['Unnamed: 0'], inplace=True)
mask = (labels_df['year'] == 2000) | (labels_df['year'] == 2002)
X_features = features_df[mask].to_numpy()
y_labels = labels_df[mask].replace({2000: 0, 2002: 1}).to_numpy().ravel()
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_features)

In [29]:
def sigmoid(z):
    return 1.0 / (1.0 + torch.exp(-z))
    
def compute_nll(predictions, targets):
    epsilon = 1e-10
    predictions = torch.clamp(predictions, epsilon, 1 - epsilon)
    return - (targets * torch.log(predictions) + (1 - targets) * torch.log(1 - predictions))

In [47]:
"""
Momentum — метод, который ускоряет обучение, помогая модели преодолевать локальные минимумы, колебания 
и медленные изменения в градиентах. Он добавляет "инерцию" в обновление весов, чтобы избежать застревания на плато 
и быстрее достичь глобального минимума

вместо простого обновления весов на основе градиента текущей итерации, 
он добавляет часть изменения из предыдущей итерации -- это позволяет моделям лучше справляться с ситуациями, 
где градиенты колеблются или изменяются медленно
"""

'\nMomentum — метод, который ускоряет обучение, помогая модели преодолевать локальные минимумы, колебания \nи медленные изменения в градиентах. Он добавляет "инерцию" в обновление весов, чтобы избежать застревания на плато \nи быстрее достичь глобального минимума\n\nвместо простого обновления весов на основе градиента текущей итерации, \nон добавляет часть изменения из предыдущей итерации -- это позволяет моделям лучше справляться с ситуациями, \nгде градиенты колеблются или изменяются медленно\n'

In [49]:
def train_neuron_momentum(X, y, w, b, lr, epochs, momentum=0.9):
    X_tensor = torch.tensor(X, dtype=torch.float64)
    y_tensor = torch.tensor(y, dtype=torch.float64)    
    w_tensor = torch.tensor(w, dtype=torch.float64)
    b_tensor = torch.tensor(b, dtype=torch.float64)
    w_vel = torch.zeros_like(w_tensor)
    b_vel = 0.0
    loss_history = []

    for epoch in range(epochs):
        total_loss = 0.0
        w_grad_sum = torch.zeros_like(w_tensor)
        b_grad_sum = 0.0
        for i in range(len(X_tensor)):
            x_sample = X_tensor[i]
            y_sample = y_tensor[i]
            z = torch.dot(w_tensor, x_sample) + b_tensor
            y_pred = sigmoid(z)
            
            loss = compute_nll(y_pred, y_sample)
            total_loss += loss.item()
            error = y_pred - y_sample
            w_grad = error * x_sample
            b_grad = error            
            w_grad_sum += w_grad
            b_grad_sum += b_grad
        
        w_grad_mean = w_grad_sum / len(X_tensor)
        b_grad_mean = b_grad_sum / len(X_tensor)
        
        w_vel = momentum * w_vel + lr * w_grad_mean
        b_vel = momentum * b_vel + lr * b_grad_mean
        
        w_tensor -= w_vel
        b_tensor -= b_vel
        
        avg_loss = total_loss / len(X_tensor)
        loss_history.append(round(avg_loss, 4))    
    final_w = [round(w.item(), 4) for w in w_tensor]
    final_b = round(b_tensor.item(), 4)    
    return final_w, final_b, loss_history

In [51]:
initial_w = np.random.uniform(-1, 1, X_normalized.shape[1])
initial_b = 0.0
lr = 0.1
epochs = 20
momentum = 0.7  

w_momentum, b_momentum, loss_momentum = train_neuron_momentum(
    X_normalized, y_labels, initial_w, initial_b, lr, epochs, momentum)
print("weights_M:", w_momentum)
print()
print("bias_M:", b_momentum)
print()
print("loss_M:", loss_momentum)

weights_M: [-0.3338, -0.3373, 0.4687, -0.8793, -0.516, 0.2786, 0.4686, 0.7099, -0.0656, 0.3466, 0.4958, 0.1603, 0.2726, -0.832, -0.5146, 0.3351, 0.4343, -0.1121, -0.2126, 0.0574, -0.009, 0.6218, 0.1545, 0.1761, -0.5171, 0.247, -0.2252, -0.4442, 0.1481, 0.0947, -0.3943, 0.0816, 0.2533, 0.163, 0.527, 0.1665, -0.6695, 0.4131, 0.5771, 0.5059, -0.1063, -0.1156, 0.4422, -0.0389, 0.3211, -0.0151, -0.3667, -0.773, 0.0108, -0.7143, -0.6759, 0.163, -0.2537, -0.1108, -0.5247, -0.3019, 0.0399, -0.7635, -0.565, -0.4119, 0.3724, -0.2494, 0.1358, -0.6232, 0.7708, -0.0226, 0.5702, 0.7382, -0.2454, 0.3907, 0.2027, 0.3046, 0.2679, 0.472, 0.4601, -0.4101, 0.1319, -0.3877, -0.5047, -0.1142, -0.2561, -0.6059, -0.5532, -0.0679, 0.1464, -0.315, -0.0639, 0.233, 0.0442, 0.1921]

bias_M: 0.077

loss_M: [1.8315, 1.8007, 1.7512, 1.693, 1.6335, 1.5772, 1.5258, 1.4789, 1.4363, 1.3961, 1.3584, 1.3235, 1.291, 1.2605, 1.2318, 1.2046, 1.1788, 1.1543, 1.1309, 1.1087]


In [53]:
predictions_momentum = sigmoid(torch.matmul(torch.tensor(X_normalized, dtype=torch.float64), torch.tensor(w_momentum, dtype=torch.float64)) + b_momentum)
rounded_predictions_momentum = torch.round(predictions_momentum)
accuracy_momentum = (rounded_predictions_momentum.numpy() == y_labels).mean()
print(f"accuracy_M: {round(accuracy_momentum * 100, 2)}%")

accuracy_M: 52.41%
